# Writing efficient Python code 

In this project, we will explore how to write a Python program that is readable while at the same time efficient with the fast runtime and the minimal memory usage. 

A part of the main Hipparcos catalog was extracted into the Hp_Sp.csv file.  
Hip_Sp.csv contains four columns: 
<ul>
      <li> Hip_No -- unique Hipparcos number </li>
      <li> Vmag -- visual magnitude as a measure of stellar apparent brightness </li> 
      <li> Mv -- absolute stellar magnitude, is a measure of the real steller brightnes and it is calculated from the Hipparcos aparent visual magnitude (Vmag) and the Hipparcos measured parallax (Plx). </li>
      <li> Spectral_type -- is a measure of stellar temperature or color. </li>
</ul>

### Importing data 

In [1]:
%%time
import numpy as np
import pandas as pd

file = '../data/Hip_Sp.csv'
new_column_names = ['Hip_No', 'Vmag', 'Mv', 'Spectral_type']
hip_sp = pd.read_csv(file, header = 0, sep=',',
                  usecols=[1,2,3,4],
                  names=new_column_names)
hip_sp.head(5)

Wall time: 1.18 s


,Hip_No,Vmag,Mv,Spectral_type
0,1,9.10,1.845016,F5
1,2,9.27,5.972221,K3V
2,3,6.61,-1.146468,B9
3,4,8.06,2.506509,F0V
4,5,8.55,0.839409,G8III


###  Pythonic .vs. non-pythonic code

How many stars from the Hipparcos catalog, are brighter than our Sun, knowing that the absolute magnitude of the Sun is 4.83? To answer this question,  we need to compare and count the number of stars from the Mv column of the hip_sp dataframe. All-stars from the catalog with the absolute magnitudes, Mv, less than 4.83, are brighter than our Sun.     